# Homework 2 (draft)

## Import:

In [79]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchmetrics import Accuracy
import tqdm
import sys
#import gymnasium as gym
import random

## Data Configuration

In [71]:
# Definisci la trasformazione per il preprocessing delle immagini
transform = transforms.Compose([
    #transforms.Resize((96, 96)),  
    transforms.Lambda(lambda x: transforms.functional.gaussian_blur(x, kernel_size=5)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),

    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizza i valori dei pixel
])
transform_test = transforms.Compose([  
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizza i valori dei pixel
])
train_folder = './train'
test_folder = './test'

train_dataset = datasets.ImageFolder(train_folder, transform=transform)
test_dataset = datasets.ImageFolder(test_folder, transform=transform_test)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


## Model Configuration

In [73]:
class SimpleCNN(torch.nn.Module):
    def __init__(self, input_channels=3):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(input_channels, 16, 5, padding=2)
        self.conv2 = torch.nn.Conv2d(16, 32, 5, padding=2)
        self.conv3 = torch.nn.Conv2d(32, 64, 5, padding=2)

        self.bn1 = torch.nn.BatchNorm2d(16)
        self.bn2 = torch.nn.BatchNorm2d(32)
        self.bn3 = torch.nn.BatchNorm2d(64)
        self.max_pool = torch.nn.MaxPool2d(2)
        #secondo parametro indica il numero di neuroni che può essere scelto in modo arbitrario 
        self.fc1 = torch.nn.Linear(64*24*24, 100)
        self.fc2 = torch.nn.Linear(100, 5)
        # il parametro è la probabilità di effetturare dropout
        self.drop = torch.nn.Dropout(0.3)

    def forward(self, x):
        x = torch.nn.functional.relu(self.bn1(self.conv1(x)))
        x = self.max_pool(x)
        x = torch.nn.functional.relu(self.bn2(self.conv2(x)))
        x = self.max_pool(x)
        x = torch.nn.functional.relu(self.bn3(self.conv3(x)))
        #x = self.max_pool(x)
        x = x.reshape((-1, 64*24*24))
        # Alternative with global average pooling: x = x.mean([2, 3])
        x = self.drop(torch.nn.functional.relu(self.fc1(x)))
        return self.fc2(x)

In [74]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cnn = SimpleCNN().to(device)

In [75]:
def accuracy(net, loader, device):
  acc = Accuracy(task='multiclass',num_classes=5).to(device)
  for xb, yb in loader:
      xb, yb = xb.to(device), yb.to(device)
      ypred = cnn(xb)
      _ = acc(ypred, yb)
  return acc.compute()

In [76]:
accuracy(cnn, test_loader, device)
loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(cnn.parameters())

## Train

In [78]:
for epoch in range(4):
  
  cnn.train()
  for xb, yb in tqdm.tqdm(train_loader):
    
    xb, yb = xb.to(device), yb.to(device)

    opt.zero_grad()
    ypred = cnn(xb)
    l = loss(ypred, yb)
    l.backward()
    opt.step()

  cnn.eval()
  print(f'Accuracy at epoch {epoch}: {accuracy(cnn, test_loader, device)}')

100%|██████████| 100/100 [01:17<00:00,  1.28it/s]


Accuracy at epoch 0: 0.6296834945678711


100%|██████████| 100/100 [01:15<00:00,  1.32it/s]


Accuracy at epoch 1: 0.6125863790512085


100%|██████████| 100/100 [01:15<00:00,  1.33it/s]


Accuracy at epoch 2: 0.6969807147979736


100%|██████████| 100/100 [01:18<00:00,  1.28it/s]


Accuracy at epoch 3: 0.679883599281311


In [ ]:
""" 
metric = Accuracy
# Define your hyperparameter search space
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]
num_epochs = 10  # Set a reasonable number of epochs

best_accuracy = 0.0
best_hyperparameters = {}

# Random search over the hyperparameter space
for _ in range(10):  # You can adjust the number of random samples
    lr = random.choice(learning_rates)
    batch_size = random.choice(batch_sizes)

    model = cnn
    opt = torch.optim.Adam(cnn.parameters(), lr=lr)

    for epoch in range(num_epochs):
        # Training loop here

    # Evaluate on the validation set
        cnn.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = cnn(inputs)
            loss = opt(outputs, labels)
            accuracy = metric(outputs, labels)
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = {'lr': lr, 'batch_size': batch_size}

print(f"Best Hyperparameters: {best_hyperparameters}")
print(f"Best Validation Accuracy: {best_accuracy}") """

In [26]:
""" import sys

try:
    import gymnasium as gym
except ModuleNotFoundError:
    print('gymnasium module not found. Try to install with')
    print('pip install gymnasium[box2d]')
    sys.exit(1)


def play(env, model):

    seed = 2000
    obs, _ = env.reset(seed=seed)
    
    # drop initial frames
    action0 = 0
    for i in range(50):
        obs,_,_,_,_ = env.step(action0)
    
    done = False
    while not done:
        p = model.predict(obs) # adapt to your model
        action = np.argmax(p)  # adapt to your model
        obs, _, terminated, truncated, _ = env.step(action)
        done = terminated or truncated




env_arguments = {
    'domain_randomize': False,
    'continuous': False,
    'render_mode': 'human'
}

env_name = 'CarRacing-v2'
env = gym.make(env_name, **env_arguments)

print("Environment:", env_name)
print("Action space:", env.action_space)
print("Observation space:", env.observation_space)

# your trained
model = cnn # your trained model

play(env, model)

 """

gymnasium module not found. Try to install with
pip install gymnasium[box2d]


AttributeError: 'tuple' object has no attribute 'tb_frame'